In [5]:
%%bash
pip install gurobipy
pip install pandas

In [1]:
import math
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
def castRisk(character):
  mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'F': 6}
  if character in mapping.keys():
    return mapping[character]

In [3]:
def distinct(d, var, num):
    modVar = []
    for index in range(num):
        if d[index]:
            modVar.append(var[index])
    occurence = {}
    for item in modVar:
        if item not in occurence.keys():
            occurence[item] = 1
        else:
            occurence[item] = occurence[item] + 1
    return len(occurence.keys())

In [4]:
def Var(x, num):
    Xsum = 0
    for num in x:
        Xsum = Xsum + num
    mean = Xsum / num
    sq_sum = 0
    for num in x:
        sq_sum = sq_sum + (num - mean)**2
    return sq_sum / num
    

In [5]:
def occurrence(d, var, num, proj, df):
    count = 0
    for index in range(num):
        if d[index]:
            if (df['Grade'][index] == proj):
                count = count + var[index]
    return count

In [6]:
# Given constants
quota = 200
penalty = 100

In [8]:
# Read Data
df = pd.read_excel('/Users/michael_khalfin/Downloads/Book6.xlsx')
df = df.dropna()

In [9]:
# Initialize model
model = gp.Model("MIP_Model")

Restricted license - for non-production use only - expires 2023-10-25


In [10]:
# Number of projects in the data table
numProjDev = df.shape[0]

In [11]:
# Add variables to the model
x = model.addVars(numProjDev, vtype=GRB.INTEGER, name="quantity")
d = model.addVars(numProjDev, vtype=GRB.BINARY, name="devTF")

In [12]:
# Initialize the objective function
obj = sum(elem1 * castRisk(elem2) for elem1, elem2 in zip(x, df['Grade'])) + penalty * Var(x, numProjDev)
model.setObjective(obj, GRB.MINIMIZE)

In [13]:
# Constraints
model.addConstr(gp.quicksum(df['Dollar'][i]*x[i] for i in range(numProjDev)) <= 10000, "price")
for i in range(numProjDev):
    model.addConstr(x[i] <= df["Quantity"][i], "quantity")
    model.addConstr(d[i] * x[i] - x[i] >= -0.5, "1st binary constraint")
    model.addConstr(d[i] * x[i] - d[i] >= -0.5, "2nd binary constraint")
    model.addConstr(x[i] / quota <= 0.05, "maximum concentration of each project")
model.addConstr(gp.quicksum(d[i] for i in range(numProjDev)) >= 20, "")
model.addConstr(distinct(d, df['Vintage'], numProjDev) >= 1, "vintage")
model.addConstr(distinct(d, df['Registry'], numProjDev) >= 1, "registry")
model.addConstr(distinct(d, df['Location'], numProjDev) >= 1, "location")
model.addConstr(distinct(d, df['Mechanism'], numProjDev) >= 1, "mechanism")
model.addConstr(distinct(d, df['Type'], numProjDev) >= 1, "typeProject")
model.addConstr(gp.quicksum(x[i] for i in range(numProjDev)) >= quota, "quota")

<gurobi.Constr *Awaiting Model Update*>

In [14]:
# Optimization
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 74 rows, 66 columns and 165 nonzeros
Model fingerprint: 0xfb82b133
Model has 66 quadratic constraints
Variable types: 0 continuous, 66 integer (33 binary)
Coefficient statistics:
  Matrix range     [5e-03, 2e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 2e+05]
  QRHS range       [5e-01, 5e-01]
Presolve removed 7 rows and 1 columns
Presolve time: 0.01s
Presolved: 67 rows, 65 columns, 227 nonzeros
Variable types: 0 continuous, 65 integer (33 binary)
Found heuristic solution: objective 10832.781250

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 10832.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.08

In [18]:
# Results
for v in model.getVars():
    print(f"{v.varName} = {v.x}")

quantity[0] = 10.0
quantity[1] = 10.0
quantity[2] = 10.0
quantity[3] = 10.0
quantity[4] = 10.0
quantity[5] = 10.0
quantity[6] = 10.0
quantity[7] = 10.0
quantity[8] = 9.0
quantity[9] = 10.0
quantity[10] = 10.0
quantity[11] = 10.0
quantity[12] = 10.0
quantity[13] = 10.0
quantity[14] = -0.0
quantity[15] = -0.0
quantity[16] = 1.0
quantity[17] = -0.0
quantity[18] = 10.0
quantity[19] = 10.0
quantity[20] = 10.0
quantity[21] = 10.0
quantity[22] = 10.0
quantity[23] = -0.0
quantity[24] = -0.0
quantity[25] = -0.0
quantity[26] = -0.0
quantity[27] = -0.0
quantity[28] = -0.0
quantity[29] = -0.0
quantity[30] = -0.0
quantity[31] = -0.0
quantity[32] = 10.0
devTF[0] = 1.0
devTF[1] = 1.0
devTF[2] = 1.0
devTF[3] = 1.0
devTF[4] = 1.0
devTF[5] = 1.0
devTF[6] = 1.0
devTF[7] = 1.0
devTF[8] = 1.0
devTF[9] = 1.0
devTF[10] = 1.0
devTF[11] = 1.0
devTF[12] = 1.0
devTF[13] = 1.0
devTF[14] = -0.0
devTF[15] = -0.0
devTF[16] = 1.0
devTF[17] = -0.0
devTF[18] = 1.0
devTF[19] = 1.0
devTF[20] = 1.0
devTF[21] = 1.0
devTF[2

In [20]:
quantity_values = [x[i].X for i in range(numProjDev)]
quantity_values

[10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 9.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 10.0]

In [29]:
# initial setup

# Initialize model
model = gp.Model("MIP_Model")

# Add variables to the model
x = model.addVars(numProjDev, vtype=GRB.INTEGER, name="quantity")
d = model.addVars(numProjDev, vtype=GRB.BINARY, name="devTF")
dbar = model.addVar(vtype=GRB.INTEGER, name="amtDevs")
v = model.addVar(vtype=GRB.INTEGER, name="amtVintage")
r = model.addVar(vtype=GRB.INTEGER, name="amtRegistry")
l = model.addVar(vtype=GRB.INTEGER, name="amtLocations")
m = model.addVar(vtype=GRB.INTEGER, name="amtMechanisms")
t = model.addVar(vtype=GRB.INTEGER, name="amtTypes")

# Initialize the objective function
obj = dbar + v + r + l + m + t
model.setObjective(obj, GRB.MAXIMIZE)

# Constraints
model.addConstr(gp.quicksum(df['Dollar'][i]*x[i] for i in range(numProjDev)) <= 10000, "price")
for i in range(numProjDev):
    model.addConstr(x[i] <= df["Quantity"][i], "quantity")
    model.addConstr(d[i] * x[i] - x[i] >= -0.5, "1st binary constraint")
    model.addConstr(d[i] * x[i] - d[i] >= -0.5, "2nd binary constraint")
    model.addConstr(x[i] / quota <= 0.05, "maximum concentration of each project")
model.addConstr(gp.quicksum(d[i] for i in range(numProjDev)) - dbar >= 0, "")
model.addConstr(distinct(d, df['Vintage'], numProjDev) - v >= 0, "vintage")
model.addConstr(distinct(d, df['Registry'], numProjDev) - r >= 0, "registry")
model.addConstr(distinct(d, df['Location'], numProjDev) - l >= 0, "location")
model.addConstr(distinct(d, df['Mechanism'], numProjDev) - m >= 0, "mechanism")
model.addConstr(distinct(d, df['Type'], numProjDev) - t >= 0, "typeProject")
model.addConstr(gp.quicksum(x[i] for i in range(numProjDev)) >= quota, "quota")

# Optimization
model.optimize()

# Results
for v in model.getVars():
    print(f"{v.varName} = {v.x}")

returnDict = {dbar.varName:dbar.X, v.varName:v.x, r.varName:r.x, l.varName:l.x, m.varName:m.x, t.varName:t.x}
returnDict

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 74 rows, 72 columns and 171 nonzeros
Model fingerprint: 0x52616252
Model has 66 quadratic constraints


Variable types: 0 continuous, 72 integer (33 binary)
Coefficient statistics:
  Matrix range     [5e-03, 2e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 2e+05]
  QRHS range       [5e-01, 5e-01]
Presolve removed 40 rows and 7 columns
Presolve time: 0.00s
Presolved: 34 rows, 65 columns, 130 nonzeros
Variable types: 0 continuous, 65 integer (33 binary)
Found heuristic solution: objective 64.0000000
Found heuristic solution: objective 74.0000000

Root relaxation: objective 7.500000e+01, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      75.0000000   75.00000  0.00%     -    0s

Explored 1 nodes (11 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available process

{'amtDevs': 33.0,
 'amtTypes': 10.0,
 'amtRegistry': 5.0,
 'amtLocations': 10.0,
 'amtMechanisms': 2.0}

In [32]:
import json
json_string = json.dumps(returnDict, indent=4, sort_keys=True)
json_string

'{\n    "amtDevs": 33.0,\n    "amtLocations": 10.0,\n    "amtMechanisms": 2.0,\n    "amtRegistry": 5.0,\n    "amtTypes": 10.0\n}'